In [1]:
import torch
from torch import nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

import datetime
# auto reolad zewnetrznych skryptow bez resetowania notebooka
import sys
sys.path.append('..')
import models
from utils.modeling_utils import SequenceGeneratorCV

%reload_ext autoreload
%autoreload 2

DEVICE = "cpu"

In [2]:
df = pd.read_csv('../../data/csv/six_cities.csv')
df['dt'] = pd.to_datetime(df['dt'])

# tylko warszawa
df = df[df['city'].isin(['Warszawa'])]
df = df.reset_index(drop=True)
#df.head()

In [3]:
# one moga zostac bez problemu jako stala czy cos, maja rozklady bardzo nienormalne
normalize_features = ['humidity', 'clouds.all', 'rain.1h', 'snow.1h', 'co', 'no', 'no2', 'so2', 'pm2_5', 'pm10', 'nh3']

# zmienne numeryczne ktore chcemy wykorzystac w modelu
numeric_features = ['day_sin', 'day_cos', 'week_sin', 'week_cos', 'month_sin', 'month_cos', 'co', 'no']

# zmienne kategoryczne ktore chcemy wykorzystac w modelu
categorical_features = ['state']

# zmienne ktorych chcemy robic predykcje
output_columns = ['co', 'no']

seq_gen = SequenceGeneratorCV(
    numeric_features = numeric_features, 
    categorical_features = categorical_features,
    output_features = output_columns,
    normalize_features = normalize_features,
    device = DEVICE 
)
seq_gen.init_preprocessor(df[:10])

In [4]:
cities_dfs, cv_indices = seq_gen.split_data(df, 5, False)
for idx_train, idx_val in cv_indices:
    df_train = pd.concat([city.loc[idx_train, :] for city in cities_dfs], axis=0)
    df_val = pd.concat([city.loc[idx_val, :] for city in cities_dfs], axis=0)
    df_train = seq_gen.preprocessor.fit_transform(df_train)
    df_val = seq_gen.preprocessor.transform(df_val)

    dataloader_train, dataloader_val = seq_gen.get_dataloaders(df_train, df_val)
    break

In [5]:
dnn_model = models.DNN_model(
    seq_gen = seq_gen,
    hidden_list=[24,32,48,96]
)

for X, y in dataloader_train:
    y_p = dnn_model(X)
    print(X.shape, y_p.shape, y.shape)
    break

torch.Size([216, 168, 9]) torch.Size([216, 48, 2]) torch.Size([216, 48, 2])


In [9]:
168*9

1512

In [8]:
[i.shape for i in dnn_model.parameters()]

[torch.Size([24, 1512]),
 torch.Size([24]),
 torch.Size([32, 24]),
 torch.Size([32]),
 torch.Size([48, 32]),
 torch.Size([48]),
 torch.Size([96, 48]),
 torch.Size([96]),
 torch.Size([96, 96]),
 torch.Size([96])]